# HOMEWORK 4 - ARNO AMABILE
___
# Assignment 1 - Distributions comparisons

In [1]:
import pylab as pl
import pandas as pd
import numpy as np
%pylab inline

import os

#this makes my plots pretty! but it is totally not mandatory to do it
import json
s = json.load( open(os.getenv ('PUI2015')+"/fbb_matplotlibrc.json") )
pl.rcParams.update(s)


Populating the interactive namespace from numpy and matplotlib



I am reading in data from January 2015. It would be a good idea to use data from warmer months, since there are more riders in the warm weather and the more data the smaller the STATISTICAL NOISE. If you are ambitios you can use data from multiple months, thus addressing systematic errors as well

In [ ]:
df=pd.read_csv(os.getenv ('PUI2015')+'/notebooks/data/201501-citibike-tripdata.csv')
print df.columns

## By Gender

In [ ]:
#df is the dataframe where the content of the csv file is stored
df['ageM'] = 2015-df['birth year'][(df['usertype'] == 'Subscriber') & (df['gender'] == 1)]
df['ageF'] = 2015-df['birth year'][(df['usertype'] == 'Subscriber') & (df['gender'] == 2)]


lets splot age in 10 year bins. the dataset is very large, so i could be split in smaller bins, but I will chose 10 years in the interest of time. if i was to do this "for real" the bin size choice should be a balance between properly sample the age space, have enough counts in each bin that the statistical noise is not significant (remember that is > sqrt(N)!) and the computational requirement to computatinal facilities ratio.
the next several steps are needed if you want to code up the KS test from scratch. that is for extra credit, so if you do not want to do it you may not need to plot split the distribution in bins and create the cumulative HOWEVER it is a great idea to do it anyways to explore your data viaually! remember Ascombe's quartet

In [ ]:
bins = np.arange(10, 99, 10)
df.ageM.groupby(pd.cut(df.ageM, bins)).agg([count_nonzero]).plot(kind='bar')
df.ageF.groupby(pd.cut(df.ageF, bins)).agg([count_nonzero]).plot(kind='bar')


this is how the cumulative distributions look like. Notice that i am normalizing them! if i want to reat an observed distribution like a probablility distribution i have to normalize it

In [ ]:
#print df.ageS, df.ageS.cumsum()

csM=df.ageM.groupby(pd.cut(df.ageM, bins)).agg([count_nonzero]).cumsum()

csF=df.ageF.groupby(pd.cut(df.ageF, bins)).agg([count_nonzero]).cumsum()

print np.abs(csM / csM.max()-csF / csF.max())

pl.plot(bins[:-1] + 5, csM / csM.max(), label = "M")
pl.plot(bins[:-1] + 5, csF / csF.max(), label = "F")
pl.legend()

they look pretty darn similar! that sets my expectation for the tests to come. if i wanted to code the KS test by hand i woud have everything i need: the normalized cumulative distributions can be subtracted from each other and the max distance can be seeked.
Notice that there amay be Nan values you are gonna have to deal with! you can do that for example with a Boolean statementsuch as df.ageF[~np.isnan(df.ageF)] or you can use numpy functions that deal with Nan values: nansum, nanmean, nanstd...
lets run the scipy KS test

In [ ]:
import scipy.stats

## KS Test to compare 2 samples
http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ks_2samp.html

In [ ]:
ks=scipy.stats.ks_2samp(df.ageM, df.ageF)


remember: the Null hypothesis is rejected if
$D_KS(n1,n2)>c(α)\sqrt{\frac{n1+n2}{n1n2}}$
where c(α) is the inverse of the KS distribution, and you do not have to know how to get that cause there are tables that list critical values!!
http://www.real-statistics.com/tests-normality-and-symmetry/statistical-tests-normality-symmetry/kolmogorov-smirnov-test/kolmogorov-distribution/
but scipy is already giving you the p-value, s you do not have to worry about it i am still giving you the table FWI and if you want to do the coding of the KS test by hand


In [ ]:
#here is the critical values tablel. Have you chosen your significance level yet?? you should do it first thing!
from IPython.display import Image
Image(filename="ks2sample_table.png")


### Pearson's test for correlation
http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr

Spearman's test for correlation
http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html#scipy.stats.spearmanr

# please perform the Spearman's test and tell me what you find
please comment on the numbers that you get in the light of the scipy manual: what is rerutned? what does it mean??
DO THE SAME FOR THE DISTRIBUTION OF RIDERS DURING THE DAY VS DURING THE NIGHT!

hint: remember how I converted the starttime last time in my own citibikes notebook into units that are easy to read. df['mystarttime'] = ... then you can use something like df['mystarttime'].dt.hour

# Assignment 2 - Goodness of fit

In [ ]:
import pylab as pl
import pandas as pd
import numpy as np
import random as rdn
%pylab inline

import os

#this makes my plots pretty! but it is totally not mandatory to do it
import json
s = json.load( open(os.getenv ('PUI2015')+"/fbb_matplotlibrc.json") )
pl.rcParams.update(s)

#plus importing scipy.stats
import scipy.stats

### 1 - Test of Binomial against Normal

In [ ]:
# Generate the distribution
N = 1000
np.random.seed(1122)
dist_n = np.random.randn(N)

alpha = 0.1
if alpha == 0.01:
    threshold_ks = 1.63/sqrt(N) 
elif alpha == 0.05:
    threshold_ks = 1.36/sqrt(N)
elif alpha >0.05:
    print "Why such low standards? We need to be more ambitious!\n"
    alpha = 0.05
    threshold_ks = 1.36/sqrt(N)
else:
    print "Why such weird standards? We cannot afford originality!\n"
    alpha = 0.05
    threshold_ks = 1.36/sqrt(N)

print "AD Test of Normal distribution on Gaussian:", \
"\n - Critical values: ", scipy.stats.anderson(dist_n, dist='norm')[1], \
"\n - Significance levels:", scipy.stats.anderson(dist_n, dist='norm')[2]    
    
threshold_ad = np.asscalar(scipy.stats.anderson(dist_n, dist='norm')[1]
                           [scipy.stats.anderson(dist_n, dist='norm')[2]==[alpha*100]])
threshold_kl = -log(alpha)/N

print "\n Critical values: KS = {0:.3f}; AD = {1:.3f}; KL = {2:.3f} \n".format(threshold_ks, threshold_ad, threshold_kl)

In [ ]:
# Create array of parameters n for the binomial distributions
np.random.seed(2415)
narray = sorted(rdn.sample(xrange(10,1000), 50))

# Create arrays of zero to fill with results
ks_b = np.zeros(len(narray))
ad_b = np.zeros(len(narray))
kl_b = np.zeros(len(narray))

# Definition of the normal distribution function to be called in KS
def mynorm (x, mu, var):
    return scipy.stats.norm.cdf(x, loc=mu, scale=var)

p=0.6 # Choice of p parameter, common to all binomial distributions

for i,n in enumerate(narray):
    # Generate the distribution
    np.random.seed(4216)
    dist_b = np.random.binomial(n, p, N)
 
    # Run the tests
    ks_b[i] = scipy.stats.kstest(dist_b, mynorm, args=(n*p, n*p*(1.0-p)))[0]
    ad_b[i] = scipy.stats.anderson(dist_b, dist='norm')[0]
    
    # For KL, I need to simulate the normal distribution as well
    mybins=np.linspace(min(dist_b),max(dist_b), 10) 
    bincenters = mybins[:-1]+0.5*(mybins[1]-mybins[0])

    kl_b [i] =  scipy.stats.entropy(np.histogram(dist_b, bins=mybins)[0], 
                                    scipy.stats.norm.pdf(bincenters, loc=n*p, scale=n*p*(1.0-p)))

In [ ]:
fig = pl.figure(figsize = (20,5))

fig.add_subplot(131)
pl.plot(narray, ks_b, label='KS')
pl.plot([narray[0], narray[-1]],[threshold_ks, threshold_ks], label="Threshold")
pl.xlabel("Parameter n")
pl.legend(loc=2)

fig.add_subplot(132)
pl.plot(narray, ad_b,  label='AD')
pl.plot([narray[0], narray[-1]],[threshold_ad, threshold_ad], label="Threshold")
pl.xlabel("Parameter n")
pl.legend(loc=2)

fig.add_subplot(133)
pl.plot(narray, kl_b, label='KL ')
pl.plot([narray[0], narray[-1]],[threshold_kl, threshold_kl], label="Threshold")
pl.xlabel("Parameter n")
pl.legend(loc=2)

**KS, AD, and KL statistics with increases of n.** Tests of divergence between samples ($N=1000$) from binomial distribution (parameters $p=0.6$ and $n$) and $\mathcal{N}(\mu=$ np$,\sigma=$ np(1-p)$)$. **Two effects: (1)** When n increases, the divergence with the normal decreases, hence the AD statistic under the threshold (fail to reject $H_0$) when n is more than 500. **(2)** As N is large, the thresholds for KS and KL are very low: even if the binomial distribution tends to the normal, the KS and KL tests "recognize" that it is not the same. The threshold for AD does not depend on N, hence the difference.

### 2 - Test of Poisson against Normal

In [ ]:
# Create array of parameters n for the binomial distributions
np.random.seed(2415)
larray = sorted(rdn.sample(xrange(1,1000), 50))

# Create arrays of zero to fill with results
ks_p = np.zeros(len(narray))
ad_p = np.zeros(len(narray))
kl_p = np.zeros(len(narray))

# Definition of the normal distribution function to be called in KS
def mynorm (x, mu, var):
    return scipy.stats.norm.cdf(x, loc=mu, scale=var)

for i,lam in enumerate(narray):
    # Generate the distribution
    np.random.seed(2118)
    dist_p = np.random.poisson(lam, N)
 
    # Run the tests
    ks_p[i] = scipy.stats.kstest(dist_p, mynorm, args=(lam, lam))[0]
    ad_p[i] = scipy.stats.anderson(dist_p, dist='norm')[0]
    
    # For KL, I need to simulate the normal distribution as well
    mybins = np.linspace(min(dist_p),max(dist_p), 10) 
    bincenters = mybins[:-1]+0.5*(mybins[1]-mybins[0])
 
    kl_p[i] = scipy.stats.entropy(np.histogram(dist_p, bins=mybins)[0],
                                  scipy.stats.norm.pdf(bincenters, loc=lam, scale=lam))

In [ ]:
fig = pl.figure(figsize = (20,5))

fig.add_subplot(131)
pl.plot(narray, ks_b, label='KS')
pl.plot([narray[0], narray[-1]],[threshold_ks, threshold_ks])
pl.xlabel(r'Parameter $\lambda$')
pl.legend(loc=2)

fig.add_subplot(132)
pl.plot(narray, ad_b,  label='AD')
pl.plot([narray[0], narray[-1]],[threshold_ad, threshold_ad])
pl.xlabel(r'Parameter $\lambda$')
pl.legend(loc=2)

fig.add_subplot(133)
pl.plot(narray, kl_b, label='KL ')
pl.plot([narray[0], narray[-1]],[threshold_kl, threshold_kl])
pl.xlabel(r'Parameter $\lambda$')
pl.legend(loc=2)

**KS, AD, and KL statistics with increases of $\lambda$.** Tests of divergence between samples ($N=1000$) from Poisson distribution (parameter $\lambda$) and $\mathcal{N}(\mu=\lambda,\sigma=\lambda)$. **Two effects: (1)** When $\lambda$ increases, the divergence with the normal decreases, hence the AD statistic under the threshold (fail to reject $H_0$) when n is more than 500. **(2)** As N is large, the thresholds for KS and KL are very low: even if the Poisson distribution tends to the normal, the KS and KL tests "recognize" that it is not the same. The threshold for AD does not depend on N, hence the difference.

In [ ]:
p = lambda x, mu : scipy.stats.distributions.poisson.pmf(x,mu)
n = lambda x, mu : scipy.stats.distributions.norm.pdf(x-mu)
c = lambda x, mu : scipy.stats.distributions.chi2.pdf(x, df=mu)
f = lambda x, mu : scipy.stats.distributions.f.pdf(x,100,100)

Dkl = lambda x ,mu, p :  p(x, mu)* np.log10(n(x, mu)) + p(x, mu) *np.log10(p(x, mu))

# Choosing parameters: the Poisson distribution needs another support x_p, as it requires integers
mu = 2
x=np.linspace(0,10,100)
x_p=np.arange(0,10,1)

pl.figure(figsize=(15,10))

pl.plot(x, n(x,mu), label='Normal')
pl.plot(x_p, p(x_p,mu), label='Poisson')
pl.plot(x_p, Dkl(x_p, mu, p), '-.', label = 'KL - Poisson')

pl.plot(x, c(x,mu), label='Chi-sq')
pl.plot(x,Dkl(x, mu, c), '--', label = 'KL - Chi-sq')

pl.plot(x, f(x,mu), label='F')
pl.plot(x,Dkl(x, mu, f), '--', label = 'KL - F')

pl.xlim(0,10)
pl.ylabel("P(k) and KL Divergence")
pl.legend( fontsize = 20)

**Kullback-Leibler Divergence statistic for several distributions.** The Divergence is a measure of the "distance" between a point in the distribution and the corresponding point from the Normal distribution with same mean and same variance.